# Analysis of the Influence of Trump's Tweets on U.S. Stock Market

## Motivation

We all know Mr. Trump loves to give his opinions on Twitter with topics covering politics, economics, entertainments, etc.

Sometimes we can see from the news titles that a certain company's stock price is influenced by Trump's opinion on Twitter. For example, when Trump scolded Toyota Motor on Twitter for planning to build plants in Mexico, Toyota’s stock price decreased for about 1%, and also when he complained about Lockheed Martin spending too much money on building F-35, its stock price dropped 2%. From these cases, we can tell that Trump’s tweets can have tremendous effects on stock market.

Therefore, we want to analyze whether Trump's tweets actually have an influence on the U.S. stock market or not.


## Data Collection


## 0. Terminology

__Trump Target Index__

- We define the companies which were mentioned in Mr.Trump's tweets "Trump Target Index". Trump target index include companies like Amazon, CNN News, The New York Times, etc.

__S&P  500__
- We collect the S&P 500 ETF stock price as the indicator of U.S. stock market.

## 1. Scrape Stock Prices

### 1) Download stock prices with pandas-datareader

Install `pandas-datareader` to download stock prices

    $ pip install pandas-datareader
    

In [40]:
import pandas as pd
import datetime as dt
import pandas_datareader as pdr

In [42]:
data_source = 'iex'
start='2017-04-15'
end = '2018-04-15'

tickers = ['WFC', 'ANTM', 'GLW', 'MRK', 'PFE', 'AMZN', 'CHTR', 'CPB', 'CSCO', 'AET',
           'HUM', 'INTC', 'JWN', 'HOG', 'DAL', 'DIS', 'FOXA', 'ETP', 'TRP', 'WMT',
           'CMCSA', 'FCAU', 'TWX', 'T', 'TM', 'S', 'GM', 'F', 'TIME', 'TWTR', 'XOM',
           'DWDP', 'LMT', 'BA', 'RXN', 'GS', 'UTX', 'JPM', 'NYT', 'GHC', 'SPY']

fail_count = 0
ok_count   = 0
results    = {}
for ticker in tickers:
    try:
        result = pdr.data.DataReader(ticker, data_source, start, end);
        results[ticker] = result
        ok_count += 1
    except:
        fail_count += 1

#print (repr(ok_count) + " loads, " + repr(fail_count) + " failures")

for key in results: # Save stock prices for future usage
    results[key].to_csv('prices/'+key+'.csv', sep=',', encoding='utf-8')

### 2) Load & format stock prices

In [43]:
def load_n_format_stock(t): # Load & format stock
    df = pd.read_csv('prices/'+t+'.csv')
    df = df[['date', 'close']]
    df['date'] = pd.to_datetime(df.date).dt.date
    df.rename(index=str, columns={'close': 'price'}, inplace=True)
    
    return df

tickers   = tickers[:-1] # Remove S&P 500 from common company list

stock_dfs = {}
for t in tickers: # Load & format company stocks
    stock_dfs[t] = load_n_format_stock(t)
        
spy_df = load_n_format_stock('spy') # Load & format S&P500

In [44]:
ticker_to_company = {'WFC': 'Wells Fargo', 'ANTM': 'Anthem', 'GLW': 'Corning',
                     'MRK': 'Merck', 'PFE': 'Pfizer', 'AMZN': 'Amazon',
                     'CHTR': 'Charter', 'CPB': 'Campbell', 'CSCO': 'Cisco',
                     'AET': 'Aetna', 'HUM': 'Humana', 'INTC': 'Intel ',
                     'JWN': 'Nordstrom', 'HOG': 'Harley-Davidson', 'DAL': 'Delta',
                     'DIS': 'Disney', 'FOXA': 'Fox', 'ETP': 'Energy Transfer',
                     'TRP': 'TransCanada', 'WMT': 'Walmart', 'CMCSA': 'Comcast',
                     'FCAU': ['Fiat', 'Chrysler'], 'TWX': ['Time Warner', 'CNN'],
                     'T': 'AT&T', 'TM': 'Toyota', 'S': 'Sprint', 
                     'GM': 'General Motors', 'F': 'Ford', 'TIME': 'TIME',
                     'TWTR': 'Twitter', 'XOM': 'Mobil ', 'DWDP': 'Dow ',
                     'LMT': 'Lockheed', 'BA': 'Boeing', 'RXN': 'Rexnord',
                     'GS': 'Goldman Sachs', 'UTX': 'United Technologies',
                     'JPM': 'JPMorgan', 'NYT': 'New York Times', 'GHC': 'Washington Post'}

def to_company(t):
    name = ticker_to_company[t]
    if isinstance(name, str):
        return name
    else: # a list of companies
        return','.join(name)

## 2. Scrape Donald Trump's Tweets

We use  [Twitter Scraper](https://github.com/bpb27/twitter_scraping)  to acquire all Trump's tweets from Apr 15th, 2017 to Apr 15th, 2018.

Required dependencies:
- selenium (3.0.1)
 - `pip3 install selenium`
- tweepy (3.5.0)
 - `pip3 install tweepy`

By following the instructions in the link above, we can get a json file which contains all tweets' information named "realdonaldtrump.json"

In [7]:
import json
import collections
import sqlite3

In [8]:
data = json.load(open('realdonaldtrump.json'))

In [9]:
data_dict = collections.defaultdict(list)
for d in data:
    data_dict['created_at'].append(d['created_at'])
    data_dict['retweet_count'].append(d['retweet_count'])
    data_dict['text'].append(d['text'])
df = pd.DataFrame(data_dict)

In [10]:
conn = sqlite3.connect(":memory:")
conn.text_factory = str
cursor = conn.cursor()
cursor.execute("""CREATE TABLE trump (
              created_at TEXT,
              retweet_count INTEGER,
              text TEXT);""")
df.to_sql('trump',con=conn,index=False,if_exists='append')

From all the tweets, select the ones that mention companies in Trump Target Index.

In [ ]:
companies = ["Wells Fargo","Merck","Pfizer","Amazon","Charter",
           "Campbell","Cisco","Aetna","Humana","Intel","Nordstrom","Harley Davidson",
           "Delta","Disney","Fox","Dakota Access","Keystone XL","Walmart","NBC",
           "Fiat","Time Warner","AT&T","Toyota","Sprint","General Motors","Ford",
           "Time Magazine","Twitter","ExxonMobil","Dow Chemical","Lockheed Martin",
           "Boeing","Rexnord","Goldman Sachs","Carrier","JPMorgan","New York Times","nytimes"]

for company in companies:
    
    query = "SELECT * FROM trump WHERE text LIKE '%"+company+"%'"
    result = cursor.execute(query)
    results = []
    for i in result:
        results.append(i)
    result_data = collections.defaultdict(list)
    for r in results:
        result_data['created_at'].append(r[0])
        result_data['retweet_count'].append(r[1])
        result_data['text'].append(r[2])
    result = pd.DataFrame(result_data)
    result.to_csv(company+'.csv',encoding='utf-8')

## Data Cleaning 

At initial attempt, the data plotted with all the tweet points on overall "Trump Index" seemed to be random without any pattern. 
Inspect into the data we found many of them are unrelated tweets, for example, the following tweet will be categorized as the company __"Intel"__, but it is actually referring to __"House Intelligence Committee"__.

<table>
<tr>
<td> <img src="intel_tweet2.png" width='400px' height='400px'/> </td>
</tr>
</table>

After data cleaning, we removed __2/3__ of the tweets, cutting down from 181 to __59__ tweets in total.

## Plot Stock Data 

### Part 1 - Overall Analysis

We first take a look at the trend of stock prices in Trump Target Index, and its relationship with the overall performance in stock market.

Here, we use S&P 500 ETF as the indicator of the U.S. stock market.

We also extract the top 5 positive and negative tweets which are retweeted for most times, to see if the positive or negative tweets have any effects on the Trump Target Index.

In [12]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import glob
import numpy as np
import plotly.plotly as py

In [13]:
init_notebook_mode(connected=True)

### Plot the trends of Trump Target Index and S&P 500

In [14]:
df_comp = pd.read_csv('company_filtered.csv', encoding ='latin1')
df_comp = df_comp.sort_values(by=['retweet_count'], ascending=False)
df_comp.created_at = pd.to_datetime(df_comp.created_at).dt.date
df_comp = df_comp.rename(index=str, columns={"created_at": "date"})

In [32]:
def get_files():
    df_s = []
    files = glob.glob("prices\*.csv")
    i = 0
    for file in files:
        if (file.endswith('DWDP.csv') or file.endswith('TIME.csv')):
            continue
        tmp_df = pd.read_csv(file)
        tmp_df['comp'] = file
        df_s.append(tmp_df)
    return df_s
    
def get_avg():
    df_s = get_files()
    
    avg_arr = []
    num_comp = len(df_s)
    i = 0
    for index, row in df_s[0].iterrows():
        sum_price = 0
        for price_df in df_s:
            #print(price_df['comp'][0])
            sum_price += price_df['close'][i]
            #price2 = df2['close'][index]
        i += 1
        avg = sum_price / num_comp
        avg_arr.append(avg)

    df_avg = pd.DataFrame(avg_arr, columns=['avg'])
    df_avg['date'] = df_s[0]['date']
    
    return df_avg

def normalize(df):
    max_p = df.max()
    #print(max_p)
    min_p = df.min()
    #print(min_p)
    diff = df - min_p
   
    diff = diff / (max_p - min_p)
    #print(diff)
    return diff
    
def get_sp500():
    df_sp = pd.read_csv("SPY.csv")
    y_diff = normalize(df_sp['close'])
    # Create a trace
    trace = go.Scatter(
        name='S&P 500',
        x=df_sp.date,
        y=y_diff,

    )
    return trace

def change_text(df):
    for text in df['text']:
        if (len(text) > 70):
            n_text = text[:71] + '<br>' + text[71:]
            print(n_text)
            return n_text
        else:
            return text

In [33]:
df_test = pd.read_csv("prices\AET.csv")
df_avg = get_avg()

df_avg.date = pd.to_datetime(df_avg.date).dt.date
df_avg.loc[:, 'avg'] = normalize(df_avg['avg'])

df_comp = df_comp.drop(df_comp.columns[df_comp.columns.str.contains('unnamed',case = False)],axis = 1)

new_df = df_avg.merge(df_comp, how='inner')

for i, text in enumerate(new_df['text']):
    if (len(text) > 70):
        n_text = text[:71] + '<br>' + text[71:]
        new_df.loc[i, 'text'] = n_text


new_df = new_df.sort_values(by=['retweet_count'], ascending=False)

neg_df = new_df[new_df['rate']=='neg']
pos_df = new_df[new_df['rate']=='pos']

In [34]:
# Create traces
trace = go.Scatter(
    name='Trump Index',
    x=df_test.date,
    y=df_avg['avg'],
)

trace_sp500 = get_sp500()

marker_trace_neg = go.Scatter(
    name='Negative Tweets',
    x=neg_df[:5]['date'],
    y=neg_df[:5]['avg'],
    mode='markers',
    marker=dict(size=10,color='#f9c172'),
    hovertext=neg_df[:5]['text'],
    textposition=["top center"],
    hoverinfo='text',
)

marker_trace_pos = go.Scatter(
    name='Positive Tweets',
    x=pos_df[:5]['date'],
    y=pos_df[:5]['avg'],
    mode='markers',
    marker=dict(size=10, color='#72f9eb'),
    hovertext=pos_df[:5]['text'],
    textposition=["top center"],
    hoverinfo='text',
)

data = [trace, marker_trace_neg, marker_trace_pos, trace_sp500]

layout = dict(
    title = "Trump Index & SP500",
    showlegend=True,
    autosize=True,
)

fig = dict(data=data, layout=layout)

iplot(fig)

#### Part 1 Summary

We can see from the above chart that the trend of Trump Index is pretty similar to the trend of S&P 500, namely the overall trend of U.S. stock market, regardless of the positive or negative tweets posted by Trump.

In the case of that the S&P 500 is going up, even though the Trump's tweets is negative, the trend of Trump Index is still going up.

It seems that it is hard to tell whether one Trump's tweet has effects on the overall Trump Index or not, because the target is just one of the Trump Index.

Plus, in long term, it looks like that the main influence on Trump Index is from the overall economic condition. 

It is hard to tell if Trump's tweets have effect on the stock prices in long term period.

Therefore, we need to look into each individual company in Trump Index to analyze the tweets effect.

### Part 2

## Load & format tweets and join with stock prices

Next, we'll load the __59__ tweets into dataframe and join with stock prices. Notice that, since some of the tweets are tweeted during weekends, there will be no matching stock price. If the resulting dataframe has empty or only one tweet, we will discard these from our dataset.

In [46]:
# Read & format all cleaned tweets (59 entries)
all_tweet_df = pd.read_csv('company_filtered.csv', encoding='utf-8')
all_tweet_df.created_at = pd.to_datetime(all_tweet_df.created_at).dt.date
all_tweet_df = all_tweet_df.rename(index=str, columns={'created_at': 'date', 'text': 'tweet'})
all_tweet_df = all_tweet_df.sort_values(by=['date'])

# Categorize tweets into each company
tweet_dfs = {}
for ticker in tickers:
    tweet_dfs[ticker] = all_tweet_df[all_tweet_df['catg'].str.contains('"'+ticker+'"')].drop(['index','catg'], axis=1)
    
# Merge with stock_dfs to get stock price on certain tweet
for key in list(tweet_dfs):
    tweet_dfs[key] = tweet_dfs[key].merge(stock_dfs[key], how='inner')
    if tweet_dfs[key].empty or tweet_dfs[key].shape[0] < 2: # drop companies with empty tweet or less than 2 tweets
        tweet_dfs.pop(key, None)
    else: # drop multiple tweet in a single day
        tweet_dfs[key] = tweet_dfs[key].drop_duplicates(['date'], keep='first')
        
# Save per company tweets info into csv (for future use)
tweet_info_list = [(key, to_company(key), len(tweet_dfs[key].index)) for key in tweet_dfs]
tweet_info_list.sort(key=lambda tup: tup[2], reverse=True)
for item in tweet_info_list:
    tweet_dfs[item[0]].to_csv('tweet_by_company/'+str(item[2])+'_'+item[0]+'.csv', sep=',', encoding='utf-8')

print ("=== Number of companies with tweets > 2: {0} ===".format(len(tweet_dfs)))
for key in tweet_dfs:
    print ("\t", key, "\t: {0} tweets".format(tweet_dfs[key].shape[0]))

=== Number of companies with tweets > 2: 8 ===
	 MRK 	: 2 tweets
	 AMZN 	: 9 tweets
	 FOXA 	: 4 tweets
	 CMCSA 	: 10 tweets
	 TWX 	: 6 tweets
	 TM 	: 3 tweets
	 NYT 	: 12 tweets
	 GHC 	: 6 tweets


## Plot all companies (stock prices v.s. tweets)

From the above, only __8__ companies remained out of __40__.
Now, let's visualize the all the remaining companies with normalized stock prices and tweets with S&P 500 as reference base line, and see if we can find something.

In [47]:
import math
from plotly import tools

In [48]:
def get_norm_data(df, ref_df=None):
    dp = df['price']
    rp = dp
    if not ref_df is None:
        rp = ref_df['price']
        
    x  = df.date
    y_norm = (dp-rp.min())/(rp.max()-rp.min())

    return (x, y_norm)

def gen_traces(stp_data, twp_data, spy_data=None, 
               tweet_point_size=12, tweet_legend_visible=False):
    trace1 = go.Scatter(x=stp_data[0], y=stp_data[1], name='stock price')
    trace2 = go.Scatter(x=twp_data[0], y=twp_data[1], 
                     visible = tweet_legend_visible,
                     opacity = 0.6,
                     marker  = dict(
                         #color = 'rgba(100, 50, 255, 0.6)',
                         size = str(tweet_point_size)
                         )
                     ,
                     mode='markers', name='tweet points')
    
    traces = [trace1, trace2]
    if spy_data is not None:
        trace3 = go.Scatter(x=spy_data[0], y=spy_data[1], opacity = 0.2,
                         marker=dict(color='rgb=(0,0,0)'), name='S&P500')
        traces.append(trace3)

    return traces

def plot_company(company, stp_data, twp_data, spy_data=None, 
                 tweet_point_size=12, tweet_legend_visible=False):
    data   = Data(gen_traces(stp_data, twp_data, spy_data, tweet_point_size, tweet_legend_visible))
    layout = Layout(title=company)
    fig    = Figure(data=data, layout=layout)
    plotly.offline.iplot(fig) 

In [51]:
def plot_all_company(stock_dfs, tweet_dfs, sub_col=2):
    titles  = []
    for t in tweet_dfs:
        titles.append(to_company(t))
    
    sub_row = math.ceil(len(titles) / sub_col)
    fig = tools.make_subplots(rows=sub_row, cols=sub_col, 
                              subplot_titles=tuple(titles))
    idx = 0
    for i,key in enumerate(tweet_dfs):
        stp_data = get_norm_data(stock_dfs[key])
        tweet_df = tweet_dfs[key]
        twp_data = get_norm_data(tweet_df, stock_dfs[key])
        spy_data = get_norm_data(spy_df)

        traces = gen_traces(stp_data, twp_data, spy_data, tweet_point_size=10, tweet_legend_visible=True)

        row_idx = idx//sub_col + 1
        col_idx = idx%sub_col + 1
        for trace in traces:
            fig.append_trace(trace, row_idx, col_idx)

        idx += 1
    fig['layout'].update(height=1024, width=1024)
    plotly.offline.iplot(fig)

In [52]:
plot_all_company(stock_dfs, tweet_dfs, sub_col=2)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]
[ (4,1) x7,y7 ]  [ (4,2) x8,y8 ]



From the above, the tweet points may seem sporadic and unrelated to stock price. This might due to the fact that the sample size is too small for statistical analysis. That said, we can still try to narrow down the problem set and see how human prediction can be achieved. 

## Human prediction on stock price with "top 5 tweets"

In this section, we pick __"top 5 tweets"__ for each company, and test how prediction could be achieved at human level. The prediction rule is simple, we anticipate per company stock price typically fluctuates proportionally to S&P 500. Thus, if Trump's tweet has effect on stock price, then the reaction of the stock price will not be proportional. Based on this assumption, if correct then mark as correct prediction, otherwise false.

In [35]:
top5_tweet_dfs = {}
for key in tweet_dfs:
    top5_tweet_dfs[key] = tweet_dfs[key].sort_values(by=['retweet_count'], ascending=False)[:5]
#plot_all_company(stock_dfs, top5_tweet_dfs, sub_col=2)

The result we get is as follows, an accuracy of __58.82%__, and __70.83%__ if outliers (CNN and NBC) removed,
since CNN and NBC are only subsidiaries of Time Warner and Comcast, the stock price composition are rather complex:

|Company|Ticker|Accuracy|Note|
|------|------|------|------|
|Amazon|AMZN|75%|(3 correct out of 5 tweets)|
|Fox|FOXA|75%|(3 correct out of 4 tweets)|
|NBC (Comcast)|CMCSA|20%|(1 correct out of 5 tweets)|
|CNN (Time Warner)|TWX|40%|(2 correct out of 5 tweets)|
|Toyota|TM|100%|(3 correct out of 3 tweets)|
|New York Times|NYT|80%|(4 correct out of 5 tweets)|
|Washington Post|GHC|60%|(3 correct out of 5 tweets)|
|Merck|MRK|50%|(1 correct out of 2 tweets)|
|__Total__||__58.82%__|(20 correct out of 34 tweets)|
|__Total (w/o CNN, NBC)__||__70.83%__|(17 correct out of 24 tweets)|

Another interesting fact we found is that most of the companies Trump focuses are the ones he dislikes, since there is little data, we will be focusing only on the __disliked company group__, specifically __Amazon__, __New York Times__, and __Washington Post__.

### Part 3- Trump's Tweets' Effect on Companies He Dislikes

From the above inspection, we found that Trump has specific hatred against the following companies
- Amazon 
- The Washington Post
- The New York Times

In [27]:
tickers = ['AMZN', 'NYT', 'GHC']
traces  = []

for ticker in tickers:
    stp_data = get_norm_data(stock_dfs[ticker])
    tweet_df = tweet_dfs[ticker]
    twp_data = get_norm_data(tweet_df, stock_dfs[ticker])
    spy_data = get_norm_data(spy_df)
    traces.extend(gen_traces(stp_data, twp_data, spy_data=None, 
                  tweet_point_size=12, tweet_legend_visible=False))

updatemenus=list([
    dict(
        active=-1,
        buttons=list([   
            dict(
                label='Amazon',
                method = 'update',
                args = [{"visible": [True,True,False,False,False,False]},
                        {"title":"Amazon"}
                       ]
            ),
            dict(
                label='NYT',
                method = 'update',
                args = [{"visible": [False,False,True,True,False,False]},
                        {"title":"The New York Times"}]
            ) ,
            dict(
                label='WAS',
                method = 'update',
                args = [{"visible": [False,False,False,False,True,True]},
                        {"title":"The Washington Post"}]
            ) 
        ]),
        direction = 'down',
        x = 0.1,
        y = 1.2,
        showactive = True, 
    ),   
])
layout = go.Layout(updatemenus=updatemenus,hovermode='closest',title='Companies',xaxis={'title':'Time',"rangeslider":dict()},
                   yaxis={'title':'Normalized Stock Price'})
# data = [trace1,trace2,trace3,trace4,trace5,trace6]
data = traces
fig = go.Figure(data=data,layout=layout)
iplot(fig)

### Amazon

From the Amazon stock price plot we can see there are many points where Trump tweeted about the company, and its stock price falled right after. 

We then pick two of the points to see what Trump has actually tweeted, one is `July 25th, 2017`, the other is `April 5th, 2018`

<table>
<tr>
<td> <img src="amazon2.png"/> </td><td> <img src="amazon(-1).png"/> </td>
</tr>
</table>

Both tweets are about Trump bashing Amazon for not paying taxes according to the regulation, and obviously the fall in the company's stock price indicates the impact Trump's tweets have had on the company itself.


### The New York Times

As for The New York Times, most points are not representative enough to say if Trump's tweets have influenced its stock price or not, however, there is one point, where Trump first called the New York Times the Failing nytimes on `July 27th, 2017`

<table>
<tr>
<td> <img src="amazon2.png" width='400px' height='400px' align='left'/> </td>
</tr>
</table>

The company's stock price dropped as much as 3% after this tweet, and this might also imply how Trump's tweets can affect company's market growth.

### The Washington Post
Last, there are a few points the stock price plummeted after Trump had tweeted about the Washington Post, and we have probed into two of them to see the exact content of the tweets. One is on `August 8th, 2017`, the other is on `December 19th, 2017`

<table>
<tr>
<td> <img src="was4.png"/> </td><td> <img src="was5.png"/> </td>
</tr>
</table>


Both are kind of related to saying the Washington Post is not telling the truth, and probably have invoked the hatred against fake news of the public, and have led to a severe drop in the Washington Post's stock price. 


## Conclusion

In conclusion, though the stock price will still follow the entire market in the long run, Trump’s tweets definitely have a certain level of impact on companies’ stock prices and cause short term fluctuations. In the future, with more data available, we anticipate to be able to predict the exact fluctuation according to the content of his tweets. 